In [6]:
import os
from pathlib import Path

import xarray as xr
import numpy as np

import requests
from tqdm.notebook import tqdm

xr.set_options(keep_attrs=True)

In [4]:
params = {
    "temperature": "t",
    "salinity": "s",
    "oxygen": "o",
    "silicate": "i",
    "phosphate": "p",
    "nitrate": "n",
}
seasons = {
    "annual": "00",
    "winter": "13",
    "spring": "14",
    "summer": "15",
    "autumn": "16",
}

data_uris = []
for param, pcode in params.items():
    for season, scode in seasons.items():
        if pcode in ["t", "s"]:
            uri = f"https://data.nodc.noaa.gov/thredds/fileServer/ncei/woa/{param}/decav/1.00/woa18_decav_{pcode}{scode}_01.nc"
        else:
            uri = f"https://data.nodc.noaa.gov/thredds/fileServer/ncei/woa/{param}/all/1.00/woa18_all_{pcode}{scode}_01.nc"
        data_uris.append(uri)

## Download WOA18 Data
This is about 4.7GB of netCDF files so it takes a while

In [5]:
def download_woa18_uris(uris):
    os.makedirs("woa18", exist_ok=True)
    for uri in uris:
        fname = uri.split("/")[-1]
        out_path = os.path.join("woa18", fname)
        
        response = requests.get(uri, stream=True)
        with tqdm.wrapattr(open(out_path, "wb"), "write",
                           miniters=1, desc=fname,
                           total=int(response.headers.get('content-length', 0))) as fout:
            for chunk in response.iter_content(chunk_size=4096):
                fout.write(chunk)
          
download_woa18_uris(data_uris)

## Preprocess files

In [12]:
root = Path("woa18")
for season, scode in seasons.items():
    os.makedirs("process", exist_ok=True)
    woa = []
    for file in root.glob(f"*{scode}_01.nc"):
        data = xr.open_dataset(file, decode_times=False)
        for var in data:
            if not var.endswith("_an"):
                continue
            woa.append(data[var].squeeze().drop("time"))
    xr.merge(woa).transpose("lon", "lat", "depth").to_netcdf(f"process/woa18_an_{scode}_01.nc")

In [15]:
def write_line(station, row):
    return (f"WOA18\t"
            f"{station+1}\t"
            f"{row.lat:.1f}\t"
            f"{row.lon:.1f}\t"
            f"{row.depth:.0f}\t"
            f"{row.t_an:.4f}\t"
            f"{row.s_an:.4f}\t"
            f"{row.o_an:.4f}\t"
            f"{row.i_an:.4f}\t"
            f"{row.n_an:.4f}\t"
            f"{row.p_an:.4f}\t"
            "\n")

JOS_HEADER = "Section\tSTATION\tLat\tLong\tPRES\tTEMP\tSALT\tO2\tSIO3\tNO3\tPO4\n"

## Request B.1
One .jos file containing the global profiles from the objectively analyzed annual fields ("an") at 1-degree latitudinal resolution but variable longitudinal resolution ("decimated"), all 102 levels.

"decimated" as follows:
* for latitudes from -60 to +60 degrees, use all 1-degree grid points
* for latitudes from -60 to -75 degrees and +60 to +75 degrees, use only every 2nd longitude
* for latitudes poleward of -75 degrees and +75 degrees, use only every 4th longitude

In [20]:
os.makedirs("jos", exist_ok=True)
with xr.open_dataset("process/woa18_an_00_01.nc") as data, open("jos/woa18_annual_decimated_all_levels.jos", 'w') as jos:
    jos.write(JOS_HEADER)
    
    # the following is a clever way of making all valid (lat, lon) pairs, it's also speedy
    points = np.array(np.meshgrid(data.coords["lat"], data.coords["lon"])).T.reshape(-1, 2)
    
    # we just need a single lat, doesn't matter which
    every_2nd_lon = data.sel(lat=0.5).lon[::2]
    every_4th_lon = data.sel(lat=0.5).lon[::4]
    
    station = 1
    for lat, lon in points:
        if abs(lat) > 75 and lon not in every_4th_lon:
            continue
        if 75 > abs(lat) > 60 and lon not in every_2nd_lon:
            continue
        profile = data.sel(lat=lat, lon=lon)
        
        if np.all(np.isnan(profile.t_an)):
            continue
        else:
            levels = np.isnan(profile["t_an"])==False
            for _, row in profile.isel(depth=levels).to_dataframe().reset_index().iterrows():
                jos.write(write_line(station, row))
            station += 1

## Request B.2
One .jos file containing the global profiles from the objectively analyzed annual fields ("an") at the same positional resolution as item B.1, but at only the 41 highlighted levels in the table above.

In [23]:
jims_depths = [
    0,
    10,
    20,
    30,
    40,
    50,
    60,
    75,
    100,
    125,
    150,
    175,
    200,
    225,
    250,
    300,
    350,
    400,
    450,
    500,
    600,
    700,
    800,
    900,
    1000,
    1200,
    1400,
    1600,
    1800,
    2000,
    2200,
    2400,
    2600,
    2800,
    3000,
    3200,
    3500,
    4000,
    4500,
    5000,
    5500,
]

os.makedirs("jos", exist_ok=True)
with xr.open_dataset("process/woa18_an_00_01.nc") as data, open("jos/woa18_annual_decimated_41_levels.jos", 'w') as jos:
    jos.write(JOS_HEADER)
    
    # the following is a clever way of making all valid (lat, lon) pairs, it's also speedy
    points = np.array(np.meshgrid(data.coords["lat"], data.coords["lon"])).T.reshape(-1, 2)
    
    # we just need a single lat, doesn't matter which
    every_2nd_lon = data.sel(lat=0.5).lon[::2]
    every_4th_lon = data.sel(lat=0.5).lon[::4]
    
    station = 1
    for lat, lon in points:
        if abs(lat) > 75 and lon not in every_4th_lon:
            continue
        if 75 > abs(lat) > 60 and lon not in every_2nd_lon:
            continue
        profile = data.sel(lat=lat, lon=lon)
        
        if np.all(np.isnan(profile.t_an)):
            continue
        else:
            levels = np.isnan(profile["t_an"])==False
            p = profile.isel(depth=levels)
            for _, row in p.where(p.depth.isin(jims_depths), drop=True).to_dataframe().reset_index().iterrows():
                jos.write(write_line(station, row))
            station += 1

## Request B.3
One .jos file containing the global heavy-decimated profiles from the objectively analyzed annual fields (an) at 2-degree latitudinal resolution and wider (than 1 & 2) variable longitudinal resolution, and at only the 41 highlighted levels in the table above. [This is a significantly smaller data file so may work better on less capable computers.]

Where "decimated" is as follows:
* use only every 2nd latitude and
* for latitudes from -30 to +30 degrees, use only every 2nd longitude
* for latitudes from -30 to -50 degrees and +30 to +50 degrees, use only every 3rd longitude
* for latitudes from -50 to -65 degrees and +50 to +65 degrees, use only every 4th longitude
* for latitudes poleward of -65 degrees and +65 degrees, use only every 5th longitude

In [25]:
os.makedirs("jos", exist_ok=True)
with xr.open_dataset("process/woa18_an_00_01.nc") as data, open("jos/woa18_annual_2deg_decimated_41_levels.jos", 'w') as jos:
    jos.write(JOS_HEADER)
    
    # the following is a clever way of making all valid (lat, lon) pairs, it's also speedy
    points = np.array(np.meshgrid(data.coords["lat"], data.coords["lon"])).T.reshape(-1, 2)
    
    # we just need a single lat, doesn't matter which
    every_2nd_lon = data.sel(lat=0.5).lon[::2]
    every_3rd_lon = data.sel(lat=0.5).lon[::3]
    every_4th_lon = data.sel(lat=0.5).lon[::4]
    every_5th_lon = data.sel(lat=0.5).lon[::5]
    
    every_2nd_lat = data.sel(lon=0.5).lat[::2]
    
    station = 1
    for lat, lon in points:
        if lat not in every_2nd_lat:
            continue
        
        if abs(lat) > 65 and lon not in every_5th_lon:
            continue
        if 65 > abs(lat) > 50 and lon not in every_4th_lon:
            continue
        if 50 > abs(lat) > 30 and lon not in every_3rd_lon:
            continue
        if 30 > abs(lat) and lon not in every_2nd_lon:
            continue
        
        profile = data.sel(lat=lat, lon=lon)
        
        if np.all(np.isnan(profile.t_an)):
            continue
        else:
            levels = np.isnan(profile["t_an"])==False
            p = profile.isel(depth=levels)
            for _, row in p.where(p.depth.isin(jims_depths), drop=True).to_dataframe().reset_index().iterrows():
                jos.write(write_line(station, row))
            station += 1

## Request B.4
Four .jos files containing the entire global decimated profiles at the resolution of items #1 And #2, for the objectively analyzed seasonal fields, for all 43 levels from the surface through 800 meters.

In [29]:
for season, scode in seasons.items():
    if scode == "00":
        continue
        
    os.makedirs("jos", exist_ok=True)
    with xr.open_dataset(f"process/woa18_an_{scode}_01.nc") as data, open(f"jos/woa18_{scode}_decimated_to_800m.jos", 'w') as jos:
        jos.write(JOS_HEADER)
        
        # the following is a clever way of making all valid (lat, lon) pairs, it's also speedy
        points = np.array(np.meshgrid(data.coords["lat"], data.coords["lon"])).T.reshape(-1, 2)
        
        # we just need a single lat, doesn't matter which
        every_2nd_lon = data.sel(lat=0.5).lon[::2]
        every_4th_lon = data.sel(lat=0.5).lon[::4]
        
        station = 1
        for lat, lon in points:
            if abs(lat) > 75 and lon not in every_4th_lon:
                continue
            if 75 > abs(lat) > 60 and lon not in every_2nd_lon:
                continue
            profile = data.sel(lat=lat, lon=lon)
            
            if np.all(np.isnan(profile.t_an)):
                continue
            else:
                levels = np.isnan(profile["t_an"])==False
                p = profile.isel(depth=levels)
                for _, row in p.where(p.depth <= 800, drop=True).to_dataframe().reset_index().iterrows():
                    jos.write(write_line(station, row))
                station += 1

## Request B.5

A library of zonal vertical sections extracted from the global profiles at 1-degree resolution, objectively analyzed annual fields (an), all 102 levels in .jos format at five-degree intervals of latitude from -80.5 degrees to +85.5 degrees. (41 sections?)

In [36]:
os.makedirs("jos", exist_ok=True)
with xr.open_dataset("process/woa18_an_00_01.nc") as data:
    
    every_5th_lat = data.sel(lon=0.5).lat[::5]
    
    for lat in every_5th_lat.values:
        data_lines = []
        
        station = 1
        profiles = data.sel(lat=lat)
        for lon in profiles.lon.values:
            profile = profiles.sel(lon=lon)
            if np.all(np.isnan(profile.t_an)):
                continue
            else:
                levels = np.isnan(profile["t_an"])==False
                for _, row in profile.isel(depth=levels).to_dataframe().reset_index().iterrows():
                    data_lines.append(write_line(station, row))
                station += 1
        if len(data_lines) > 0:
            with open(f"jos/woa18_lat_{lat}.jos", 'w') as jos:
                jos.write(JOS_HEADER)
                jos.write("".join(data_lines))

## Request B.6
A library of meridional vertical sections extracted from the global profiles at 1-degree resolution, objectively analyzed annual fields (an), all 102 levels in .jos format in .jos format at 10-degree intervals of longitude from +0.5 degrees to -0.5 degrees. (36 sections?)

In [37]:
os.makedirs("jos", exist_ok=True)
with xr.open_dataset("process/woa18_an_00_01.nc") as data:
    
    every_5th_lon = data.sel(lat=0.5).lon[::5]
    
    for lon in every_5th_lon.values:
        data_lines = []
        
        station = 1
        profiles = data.sel(lon=lon)
        for lat in profiles.lat.values:
            profile = profiles.sel(lat=lat)
            if np.all(np.isnan(profile.t_an)):
                continue
            else:
                levels = np.isnan(profile["t_an"])==False
                for _, row in profile.isel(depth=levels).to_dataframe().reset_index().iterrows():
                    data_lines.append(write_line(station, row))
                station += 1
        if len(data_lines) > 0:
            with open(f"jos/woa18_lon_{lon}.jos", 'w') as jos:
                jos.write(JOS_HEADER)
                jos.write("".join(data_lines))